In [2]:
import sys
sys.path.append("../src/")
sys.path.append("../")



import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [3]:
bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmp5udk88ut
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [85]:
"""
take a look at the eval words
"""
pd.set_option('display.max_colwidth', None)
eval_words = pd.read_csv('../data/processed/feature_annotation_pilot_annotation_items.csv')
eval_words

,sentenceid,lemma,word,literal?,pos,construction,normalized_construction,sentence,corpus,info,genre,source,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,collapse,collapse,literal,n,ADJ X,modifier_x,"The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods.",coca,COCA:2012:BLOG,yankeeanalysts.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,collapse,collapse,literal,n,colapse of X,prep_complement,Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.,coca,COCA:2012:BLOG,freethoughtblogs.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,collapse,collapse,literal,n,collapse of X,prep_complement,"Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen.",coca,2012,BLOG,...square.blogs.cnn.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,collapse,collapse,literal,n,collapse of X,prep_complement,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,coca,1998,MAG,Cosmopolitan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,collapse,collapse,literal,n,NOUN collapse,compound,"After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated outside. The safest place to sleep is out in the open.",coca,2018,SPOK,PBS_Newshour,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,collapse,collapse,literal,n,NOUN collapse,compound,"By now, everybody has heard about the plight of 33 men trapped deep underground in Chile by a mine collapse and that they're going to have to wait for months yet before rescue.",coca,2010,SPOK,NPR_TalkNation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,collapse,collapsed,literal,v,collapse PREP,verb,"Veins bulging and throbbing, he struggled for breath and collapsed on the floor.",coca,FIC: Goddess of fire,2017,Bharti Kirchner,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,collapse,collapse,literal,v,collapse PREP,verb,Roads may have weakened and could collapse under the weight of a car.,coca,COCA:2012:WEB,ready.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,collapse,collapse,literal,v,collapse PREP,verb,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",coca,COCA:2003:FIC,MassachRev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,collapse,collapse,metaphorical,n,collapse of X,prep_complement,"The collapse of the Mobutu dictatorship led to a collision between two forms of opposition to Mobutu, the internal opposition exemplified by Tshisekedi, committed to non-violent direct action, and the armed opposition, based abroad, exemplified by Kabila.",mohler_2015,05 Engineering and Business: Engineering,Built,dictatorship,NaN,collapse,dictatorship,Government,collapse,Building,7.213e+05


In [45]:
long_form = [
    'Features [Vision - Vision]',
'Features [Vision - Bright]',
'Features [Vision - Dark]',
'Features [Vision - Color]',
'Features [Vision - Pattern]',
'Features [Vision - Large]',
'Features [Vision - Small]',
'Features [Vision - Motion]',
'Features [Vision - Biomotion]',
'Features [Vision - Fast]',
'Features [Vision - Slow]',
'Features [Vision - Shape]',
'Features [Vision - Face]',
'Features [Vision - Body]',
'Features [Somatic - Touch]',
'Features [Somatic - Temperature]',
'Features [Somatic - Texture]',
'Features [Somatic - Weight]',
'Features [Somatic - Pain]',
'Features [Audition - Audition]',
'Features [Audition - Loud]',
'Features [Audition - Low]',
'Features [Audition - High]',
'Features [Audition - Sound]',
'Features [Audition - Music]',
'Features [Audition - Speech]',
'Features [Gustation - Taste]',
'Features [Olfaction - Smell]',
'Features [Motor - Head]',
'Features [Motor - UpperLimb]',
'Features [Motor - LowerLimb]',
'Features [Spatial - Landmark]',
'Features [Spatial - Path]',
'Features [Spatial - Scene]',
'Features [Spatial - Near]',
'Features [Spatial - Toward]',
'Features [Spatial - Away]',
'Features [Spatial - Number]',
'Features [Temporal - Time]',
'Features [Temporal - Duration]',
'Features [Temporal - Long]',
'Features [Temporal - Short]',
'Features [Causal - Consequential]',
'Features [Social - Social]',
'Features [Social - Human]',
'Features [Social - Communication]',
'Features [Social - Self]',
'Features [Cognition - Cognition]',
'Features [Emotion - Benefit]',
'Features [Emotion - Harm]',
'Features [Emotion - Pleasant]',
'Features [Emotion - Unpleasant]',
'Features [Emotion - Happy]',
'Features [Emotion - Sad]',
'Features [Emotion - Angry]',
'Features [Emotion - Disgusted]',
'Features [Emotion - Fearful]',
'Features [Emotion - Surprised]',
'Features [Drive - Drive]',
'Features [Drive - Needs]',
'Features [Attention - Attention]',
'Features [Attention - Arousal]'
]

feature_categories = dict()

for col in long_form:
    feature = col.split()[-1][:-1]
    category = col.split()[1][1:]
    feature_categories[feature] = category

features = feature_categories.keys()

feature_categories

{'Vision': 'Vision',
 'Bright': 'Vision',
 'Dark': 'Vision',
 'Color': 'Vision',
 'Pattern': 'Vision',
 'Large': 'Vision',
 'Small': 'Vision',
 'Motion': 'Vision',
 'Biomotion': 'Vision',
 'Fast': 'Vision',
 'Slow': 'Vision',
 'Shape': 'Vision',
 'Face': 'Vision',
 'Body': 'Vision',
 'Touch': 'Somatic',
 'Temperature': 'Somatic',
 'Texture': 'Somatic',
 'Weight': 'Somatic',
 'Pain': 'Somatic',
 'Audition': 'Audition',
 'Loud': 'Audition',
 'Low': 'Audition',
 'High': 'Audition',
 'Sound': 'Audition',
 'Music': 'Audition',
 'Speech': 'Audition',
 'Taste': 'Gustation',
 'Smell': 'Olfaction',
 'Head': 'Motor',
 'UpperLimb': 'Motor',
 'LowerLimb': 'Motor',
 'Landmark': 'Spatial',
 'Path': 'Spatial',
 'Scene': 'Spatial',
 'Near': 'Spatial',
 'Toward': 'Spatial',
 'Away': 'Spatial',
 'Number': 'Spatial',
 'Time': 'Temporal',
 'Duration': 'Temporal',
 'Long': 'Temporal',
 'Short': 'Temporal',
 'Consequential': 'Causal',
 'Social': 'Social',
 'Human': 'Social',
 'Communication': 'Social',
 'Se

In [46]:
# model dimensions dont have full 65 oops who knows what happened if our original downloaded data was bunk
len(feature_categories)

62

In [47]:
model = torch.load('../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300')

In [87]:
predictions = []
for index, row in eval_words.iterrows():
        
    # predict features in context
#     singular = row.word
#     past = singular+"d"
#     try:
#         prediction = model.predict_in_context(singular, row.sentence, bert)
#         word = singular
#     except:
#         prediction = model.predict_in_context(past, row.sentence, bert)
#         word = past

    print(row.word)
    print(row.sentence)
    prediction = model.predict_in_context(row.word, row.sentence, bert)

    
    predictions.append(prediction)
    
eval_words['predictions'] = predictions

collapse
The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods. 
collapse
Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.
collapse
 Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen. 
collapse
Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments. 
collapse
 After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated 

flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a role in stanching the flow of guns from traffickers to criminals.
flow
There will be a much larger flow of taxes going into the governmental coffers.
flow
Distance and Open Learni

In [88]:
eval_words

,sentenceid,lemma,word,literal?,pos,construction,normalized_construction,sentence,corpus,info,genre,source,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,predictions
0,1,collapse,collapse,literal,n,ADJ X,modifier_x,"The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods.",coca,COCA:2012:BLOG,yankeeanalysts.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.17007482, -1.0727395, -0.035421513, -1.7673836, -0.72395176, 1.6127343, -0.57737005, 1.0312792, 0.29067618, 0.92168635, 0.3940181, -0.3422855, -0.071819924, 0.6122407, -0.7304631, -0.49886683, -1.1563692, -0.5102963, 2.863563, 0.63915074, 0.9059837, -0.06645742, 0.09419241, 0.1772581, -1.0314965, 0.20641339, -0.93982416, -0.7926283, -0.731561, -0.7462429, 1.4663396, 0.53554666, 1.2995608, 0.4478942, -0.67584836, 0.031571615, 0.9471882, 1.6619941, 0.43925083, 1.501053, 1.7965709, 1.7368805, 3.9514933, 1.7608931, 0.52988183, 0.5640038, 0.23374997, 1.9130355, 0.05008972, 3.2751575, -1.8313127, 4.3908725, -1.4061807, 2.8423026, 2.5699995, 1.8090757, 2.879613, 2.4526093, 1.4197265, -0.26530486, 2.5662663, 2.3884003]"
1,2,collapse,collapse,literal,n,colapse of X,prep_complement,Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.,coca,COCA:2012:BLOG,freethoughtblogs.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1.8430359, 0.23360011, 0.66594195, -0.26582646, 0.22100092, 2.2082002, 0.60315835, 2.1849763, 0.31396225, 2.1533887, 0.50645274, 1.2530856, -0.5885095, -0.03985627, 0.17683247, 1.24571, -0.25553998, 0.7395566, 2.8395622, 2.4095562, 3.0292993, 0.7288414, 1.0904317, 2.0684638, -0.09909324, -0.40685183, -0.16086887, -0.044363946, -0.44005936, 0.75343555, 0.09581741, -0.29327437, 1.9847698, 0.6519944, -0.051884353, 0.76009154, 1.3821492, 1.5852051, 1.0521502, 2.1440248, 1.5551548, 2.677025, 5.097057, 0.85726434, 0.18523878, 0.46898296, -0.12475592, 0.8166257, 0.57501626, 4.282963, -1.1546761, 4.265228, -0.93719786, 2.4769628, 2.105694, 1.4451896, 3.5663216, 3.401918, 1.7431418, 0.17303732, 4.133538, 3.258229]"
2,3,collapse,collapse,literal,n,collapse of X,prep_complement,"Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen.",coca,2012,BLOG,...square.blogs.cnn.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[3.1374872, 0.58411777, 1.5429802, 0.85693645, 0.93377143, 2.9551246, 0.5128914, 2.1863537, -0.11705746, 2.0338528, 0.59236443, 1.979711, -0.26123992, -0.24761932, 0.025082458, 1.6251247, 0.18467087, 1.4367735, 3.7274303, 1.8423178, 2.7029, 0.53224003, 0.5716734, 1.6390349, -0.4801493, -0.19355708, 0.002665475, 0.5401312, -0.57339764, 0.2791911, 0.48115447, 0.57623357, 1.933609, 1.8463944, -0.5268037, 0.77156633, 1.3145041, 1.4216013, 1.504519, 2.5242004, 1.9433563, 2.3446128, 5.2803316, 1.0698758, -0.13112667, -0.21280384, -1.191257, 0.22081725, -0.3628055, 5.393575, -1.6718326, 5.6208076, -1.5897218, 3.6178179, 3.017824, 2.2286232, 4.621779, 3.2315338, 1.4069544, -0.61182183, 4.2393737, 3.6311076]"
3,4,collapse,collapse,literal,n,collapse of X,prep_complement,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,coca,1998,MAG,Cosmopolitan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.23787984, -1.1537052, 0.100213945, -1.8981329, -0.75913787, 0.4577593, 0.10598539, 1.0122651, 0.20712434, 0.93216133, 0.41344792, -0.48369926, 0.16174042, 0.88114643, -0.035644848, 0.14802553, -0.94850206, -0.6858958, 3.5241065, 0.92757046, 1.123782, -0.06269631, 0.27046028, 0.4667408, -0.80289537, 0.36579305, -0.5838674, -0.6772649, 0.16646565, 0.5819527, 1.2838995, -0.274096

In [50]:
print(eval_words.lemma.unique())

['collapse' 'cure' 'flow']


In [89]:
len(eval_words)

57

In [ ]:
# metaphor analysis

In [61]:
deltas = []

for lemma in eval_words.lemma.unique():
    preds = eval_words[eval_words['word'] == lemma]
    lit = preds[preds['literal?'] == 'literal']
    met = preds[preds['literal?'] == 'metaphorical']

    lit_avg = np.average(met.predictions)
    met_avg = np.average(lit.predictions)
    delta = lit_avg - met_avg
    deltas.append(delta)
    
    print(lemma)
    print(dict(zip(features,delta)))
    
print("average delta from literal to metaphorical across lemmas")
avg_delta = np.average(deltas, axis = 0)
print(dict(zip(features,avg)))


collapse
{'Vision': -0.17419362, 'Bright': -0.04469891, 'Dark': 0.28317934, 'Color': 0.2153717, 'Pattern': -0.31086218, 'Large': 0.15614796, 'Small': -0.21141703, 'Motion': -0.29308677, 'Biomotion': 0.20201746, 'Fast': -0.29732418, 'Slow': -0.123974174, 'Shape': -0.3082443, 'Face': 0.71522945, 'Body': 0.113982856, 'Touch': -0.96647584, 'Temperature': -0.3126899, 'Texture': -0.41451985, 'Weight': -0.019477904, 'Pain': -0.26177907, 'Audition': 0.35292113, 'Loud': 0.11491084, 'Low': 0.12656254, 'High': 0.102764726, 'Sound': 0.048571587, 'Music': 0.09827435, 'Speech': 0.8230707, 'Taste': -0.07647213, 'Smell': -0.14370206, 'Head': 0.4040456, 'UpperLimb': -0.5115238, 'LowerLimb': -0.79708487, 'Landmark': -0.3484, 'Path': -0.3406272, 'Scene': -0.22805297, 'Near': -0.61971045, 'Toward': -0.2980911, 'Away': -0.3730873, 'Number': 0.15924025, 'Time': 0.26727223, 'Duration': 0.0844177, 'Long': 0.13028455, 'Short': -0.27401423, 'Consequential': 0.21458769, 'Social': 0.399069, 'Human': 0.6568152, 'C

In [73]:
# so we want to take each category
coarse_categories = set(feature_categories.values())

In [84]:
results = dict()
for coarse_cat in coarse_categories:
    print(coarse_cat)
    delta_results = []
    for delta in deltas:

        delta = dict(zip(features,delta))
        #print(delta)

        res = []
        for key, value in delta.items():
            if feature_categories[key] == coarse_cat:
                #print(coarse_cat)
                #print(value)
                res.append(value)
        #print(len(res))
        #print(res)
        res = np.average(res)
        #print(res)
        delta_results.append(res)
    results[coarse_cat] = np.average(delta_results)
print(results)

Causal
Vision
Social
Cognition
Emotion
Drive
Attention
Spatial
Audition
Gustation
Somatic
Temporal
Motor
Olfaction
{'Causal': 0.19326107, 'Vision': 0.11247742, 'Social': 0.38922533, 'Cognition': 0.37266442, 'Emotion': 0.08369267, 'Drive': 0.002976517, 'Attention': 0.3584292, 'Spatial': -0.08276116, 'Audition': 0.26506618, 'Gustation': -0.41787037, 'Somatic': -0.27161542, 'Temporal': 0.01405403, 'Motor': -0.13316245, 'Olfaction': -0.124383934}


# construction analysis

hypothesis: nouns in ASNs (argument supporting constructions) will behave more like metaphorical language in emphasizing abstract features. and nouns in compounds will behave more liek literal uses, with emphasis of physical features

In [91]:
# check that we've got this label coded correctly
eval_words.normalized_construction.unique()

array(['modifier_x', 'prep_complement', 'compound', 'verb'], dtype=object)

In [98]:
const_deltas = []

for lemma in eval_words.lemma.unique():
    preds = eval_words[eval_words['word'] == lemma]
    lit = preds[preds['normalized_construction'] == 'compound']
    met = preds[preds['normalized_construction'] == 'prep_complement']

    lit_avg = np.average(met.predictions)
    met_avg = np.average(lit.predictions)
    delta = lit_avg - met_avg
    const_deltas.append(delta)
    
    print(lemma)
    print(dict(zip(features,delta)))
    
print("average delta from compound (lit) to prep complement (met) across lemmas")
avg_delta = np.average(deltas, axis = 0)
print(dict(zip(features,avg)))

collapse
{'Vision': -0.49171996, 'Bright': -0.3067559, 'Dark': -0.42609853, 'Color': -0.7228577, 'Pattern': -0.5045803, 'Large': -0.71385074, 'Small': 0.09049165, 'Motion': -0.31519198, 'Biomotion': 0.27527595, 'Fast': -0.26784182, 'Slow': -0.22224322, 'Shape': -0.58232665, 'Face': 0.52744395, 'Body': 0.5994586, 'Touch': 0.031878203, 'Temperature': -0.4736963, 'Texture': -0.2786922, 'Weight': -0.51478946, 'Pain': -0.35493445, 'Audition': 0.312212, 'Loud': -0.015454292, 'Low': 0.01142323, 'High': 0.19196993, 'Sound': 0.06910312, 'Music': 0.3025833, 'Speech': 0.6156839, 'Taste': -0.09012462, 'Smell': -0.3605376, 'Head': 0.45471847, 'UpperLimb': 0.707011, 'LowerLimb': -0.0066573024, 'Landmark': -0.38008493, 'Path': -0.25892484, 'Scene': -0.42575854, 'Near': 0.17477798, 'Toward': -0.055330873, 'Away': -0.044544935, 'Number': -0.1135366, 'Time': -0.11053038, 'Duration': -0.096343994, 'Long': -0.21440649, 'Short': 0.1968143, 'Consequential': -0.0645628, 'Social': 0.46081984, 'Human': 0.86238

In [100]:
results = dict()
for coarse_cat in coarse_categories:
    print(coarse_cat)
    delta_results = []
    for delta in const_deltas:

        delta = dict(zip(features,delta))
        #print(delta)

        res = []
        for key, value in delta.items():
            if feature_categories[key] == coarse_cat:
                #print(coarse_cat)
                #print(value)
                res.append(value)
        #print(len(res))
        #print(res)
        res = np.average(res)
        print(res)
        delta_results.append(res)
    results[coarse_cat] = np.average(delta_results)
#print(results)

Causal
-0.0645628
-0.38981843
-0.16397333
Vision
-0.21862833
0.12004639
0.31502256
Social
0.5642164
-0.1222862
0.28738648
Cognition
0.5416862
0.015124083
0.17980576
Emotion
-0.12432009
-0.27671796
-0.08532443
Drive
0.20570996
0.28784436
-0.07900095
Attention
-0.102904916
-0.6068026
0.060249925
Spatial
-0.15762897
0.010953302
0.103421904
Audition
0.21250303
-0.4132634
0.12924932
Gustation
-0.09012462
0.06645012
-0.2676504
Somatic
-0.31804684
-0.022410352
-0.107581474
Temporal
-0.05611664
-0.28529233
-0.075606644
Motor
0.38502407
-0.22612226
0.071339406
Olfaction
-0.3605376
-0.07619703
-0.48292324
